<a href="https://colab.research.google.com/github/Is7ac/ML/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset_dir = "/content/drive/MyDrive/chest_xray/train"

In [4]:
# Step 2: Image Preprocessing

# Define a function to preprocess images
def preprocess_image(image_path, target_size):
    image = cv2.imread(image_path)
    image = cv2.resize(image, target_size)
    image_tensor = torch.tensor(image, dtype=torch.float32)  # Convert to tensor
    return image_tensor

In [7]:
# Define target image size
target_size = (128, 128)

# Load images and labels
images = []
labels = []

for file_name in os.listdir(dataset_dir):
    file_dir = os.path.join(dataset_dir, file_name)
    for class_name in os.listdir(dataset_dir):
      class_dir = os.path.join(dataset_dir, class_name)
      for image_name in os.listdir(class_dir):
          image_path = os.path.join(class_dir, image_name)
          image_tensor = preprocess_image(image_path, target_size)
          images.append(image_tensor)
          labels.append(class_name)  # Assuming folder names are class labels
# Convert string labels to numerical labels
label_encoder = LabelEncoder()
numerical_labels = label_encoder.fit_transform(labels)

# Convert numerical labels to tensor
# Convert lists to tensors
images = torch.stack(images)
labels = torch.tensor(numerical_labels)

In [8]:
# Step 3: Classification - Fully Connected Neural Network

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [9]:
# Define the neural network manually
class FullyConnectedNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
        super(FullyConnectedNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input tensor
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [13]:
# Define model parameters
input_size = np.prod(target_size)
hidden_size1 = 128
hidden_size2 = 64
num_classes = len(np.unique(labels))

# Create an instance of the model
model = FullyConnectedNN(input_size, hidden_size1, hidden_size2, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Define batch size and number of epochs
batch_size = 32
epochs = 10

# Train the model
for epoch in range(epochs):
    for i in range(0, len(X_train), batch_size):
        inputs = X_train[i:i+batch_size]
        targets = y_train[i:i+batch_size]

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).sum().item() / len(y_test)
    print(f'Test Accuracy: {accuracy:.4f}')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x49152 and 16384x128)